In [1]:
import itertools
import sys
import os
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchinfo

In [4]:
from PotteryDataset import create_pottery_dataloaders, create_pottery_datasets, feature_types, feature_type_combos
from PotteryChronologyPredictor import PotteryChronologyPredictor, train

sys.path.append(os.path.abspath(".."))
from utils import read_features, read_targets, print_info_features, print_info_targets, train_val_split, get_dimensions, \
    metrics_r

## Read Features and Targets

In [5]:
path = os.path.abspath(os.path.join(os.getcwd(), "../../data/chronology_prediction"))

In [6]:
targets = ["StartYear", "YearRange"]

In [7]:
X = read_features(path, f_type="tensors")
y = read_targets(path, targets, f_type="tensors")

Loaded X_train_tfidf
Loaded X_train_bert
Loaded X_train_cannyhog
Loaded X_train_resnet
Loaded X_train_vit
Loaded X_test_tfidf
Loaded X_test_bert
Loaded X_test_cannyhog
Loaded X_test_resnet
Loaded X_test_vit
Loaded y_train
Loaded y_test


In [8]:
print_info_features(X)

{
	train: {
		tfidf: 
			<class 'torch.Tensor'>
			shape = torch.Size([1719, 300]), 
		bert: 
			<class 'torch.Tensor'>
			shape = torch.Size([1719, 768]), 
		cannyhog: 
			<class 'torch.Tensor'>
			shape = torch.Size([1719, 2917]), 
		resnet: 
			<class 'torch.Tensor'>
			shape = torch.Size([1719, 2048]), 
		vit: 
			<class 'torch.Tensor'>
			shape = torch.Size([1719, 768]), 
	},
	test: {
		tfidf: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 300]), 
		bert: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 768]), 
		cannyhog: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 2917]), 
		resnet: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 2048]), 
		vit: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 768]), 
	},
}


In [9]:
print_info_targets(y)

{
	train: 
		<class 'torch.Tensor'>
		shape   = torch.Size([1719, 2])
	test: 
		<class 'torch.Tensor'>
		shape   = torch.Size([191, 2])
}


## Train-Validation Split

In [10]:
X, y = train_val_split(X, y)

In [11]:
print_info_features(X)

{
	train: {
		tfidf: 
			<class 'torch.Tensor'>
			shape = torch.Size([1547, 300]), 
		bert: 
			<class 'torch.Tensor'>
			shape = torch.Size([1547, 768]), 
		cannyhog: 
			<class 'torch.Tensor'>
			shape = torch.Size([1547, 2917]), 
		resnet: 
			<class 'torch.Tensor'>
			shape = torch.Size([1547, 2048]), 
		vit: 
			<class 'torch.Tensor'>
			shape = torch.Size([1547, 768]), 
	},
	val: {
		tfidf: 
			<class 'torch.Tensor'>
			shape = torch.Size([172, 300]), 
		bert: 
			<class 'torch.Tensor'>
			shape = torch.Size([172, 768]), 
		cannyhog: 
			<class 'torch.Tensor'>
			shape = torch.Size([172, 2917]), 
		resnet: 
			<class 'torch.Tensor'>
			shape = torch.Size([172, 2048]), 
		vit: 
			<class 'torch.Tensor'>
			shape = torch.Size([172, 768]), 
	},
	test: {
		tfidf: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 300]), 
		bert: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 768]), 
		cannyhog: 
			<class 'torch.Tensor'>
			shape = torch.Size([191, 2917]), 
		resnet: 
		

In [12]:
print_info_targets(y)

{
	train: 
		<class 'torch.Tensor'>
		shape   = torch.Size([1547, 2])
	val: 
		<class 'torch.Tensor'>
		shape   = torch.Size([172, 2])
	test: 
		<class 'torch.Tensor'>
		shape   = torch.Size([191, 2])
}


## Dimensions

In [13]:
X_dim, y_dim = get_dimensions(X, y)


X Dimensions: {'tfidf': 300, 'bert': 768, 'cannyhog': 2917, 'resnet': 2048, 'vit': 768}
y Dimensions: 2


## Torch Datasets and Dataloaders

In [14]:
datasets = create_pottery_datasets(X, y)
loaders = create_pottery_dataloaders(datasets)

## Initialize Model



In [15]:
hidden_size = 512

In [16]:
models = {
             ft: PotteryChronologyPredictor([X_dim[ft]], y_dim, hidden_size, chronology_target="years")
             for ft in feature_types
         } | {
             ft_txt + " + " + ft_img: PotteryChronologyPredictor([X_dim[ft_txt], X_dim[ft_img]], y_dim, hidden_size, chronology_target="years")
             for ft_txt, ft_img in feature_type_combos
         }

In [17]:
for ft, model in models.items():
    print(f"\n\nMODEL SUMMARY - FEATURE TYPE: {ft.upper()}")
    model.summary()



MODEL SUMMARY - FEATURE TYPE: TFIDF
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
PotteryChronologyPredictor               [1, 300]                  [1, 2]                    --                        --
├─ModuleList: 1-1                        --                        --                        --                        --
│    └─Sequential: 2-1                   [1, 300]                  [1, 512]                  --                        --
│    │    └─Linear: 3-1                  [1, 300]                  [1, 512]                  154,112                   154,112
│    │    └─ReLU: 3-2                    [1, 512]                  [1, 512]                  --                        --
│    │    └─Dropout: 3-3                 [1, 512]                  [1, 512]                  --                        --
├─Sequential: 1-2                        [1, 512]                  [1, 2]                    -- 

## Model Training

In [18]:
criterion = nn.MSELoss()
metrics = {m: metrics_r[m] for m in metrics_r.keys() if m in ["mae", "r2", "medae"] }

In [19]:
for ft, model in models.items():
    if ft != "bert": continue
    models[ft] = train(model, loaders["train"][ft], loaders["val"][ft], criterion, metrics)

Epoch 001 | Train Loss: 56270.3576 | Val. Loss: 16575.0605 | mae: 93.9986 | r2: -4.9168 | medae: 103.1483
Epoch 002 | Train Loss: 7868.3684 | Val. Loss: 3868.3706 | mae: 43.7998 | r2: -0.2102 | medae: 40.7829
Epoch 003 | Train Loss: 4950.1287 | Val. Loss: 3934.1294 | mae: 44.1706 | r2: -0.1989 | medae: 43.6453
Epoch 004 | Train Loss: 4764.5494 | Val. Loss: 3581.2771 | mae: 42.5111 | r2: -0.1215 | medae: 37.2968
Epoch 005 | Train Loss: 4120.6580 | Val. Loss: 3528.0732 | mae: 41.8559 | r2: -0.0769 | medae: 37.8410
Epoch 006 | Train Loss: 4213.9272 | Val. Loss: 3420.5710 | mae: 40.7167 | r2: -0.0578 | medae: 33.1485
Epoch 007 | Train Loss: 4050.6586 | Val. Loss: 3497.8779 | mae: 41.5961 | r2: -0.0962 | medae: 40.8312
Epoch 008 | Train Loss: 3923.0477 | Val. Loss: 3194.4341 | mae: 39.6375 | r2: -0.0178 | medae: 32.0437
Epoch 009 | Train Loss: 3571.4920 | Val. Loss: 3748.9880 | mae: 41.8721 | r2: -0.1283 | medae: 39.2939
Epoch 010 | Train Loss: 3577.7723 | Val. Loss: 3034.5662 | mae: 38.858